### Imports

In [5]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from pathlib import Path


### Load Datasets

In [6]:
data_path = Path.cwd()

# Pfad zur CSV-Datei das Später erstellt wird
train = pd.read_csv(data_path/'../csv/feature_data/train_engineered.csv')
test = pd.read_csv(data_path/'../csv/feature_data/test_engineered.csv')


In [7]:
print(len(train))
print(len(test))

891
418


### Modell:  XGBoost-Klassifikator

In [8]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from pathlib import Path


data_path = Path.cwd()

# Pfad zur CSV-Datei das Später erstellt wird
train = pd.read_csv(data_path/'../csv/feature_data/train_engineered.csv')
test = pd.read_csv(data_path/'../csv/feature_data/test_engineered.csv')
print(len(train))
print(len(test))

# Angenommen, deine Trainings- und Testdatensätze wurden bereits geladen in `train` und `test`

# Setze 'Survived' im Testdatensatz auf -1, um die Struktur beizubehalten
test['Survived'] = -1

# Entferne die 'Ticket' Spalte aus beiden Datensätzen
# train = train.drop(['Ticket'], axis=1)
# test = test.drop(['Ticket'], axis=1)

# Definiere Features und Label
X_train = train.drop(['PassengerId', 'Survived'], axis=1)
y_train = train['Survived']

# Vorbereitung des Modells
# Hier verwenden wir XGBClassifier direkt, da XGBoost seine eigene Methode zur Handhabung von Feature-Skalierung hat,
# und es ist nicht zwingend notwendig, StandardScaler in der Pipeline zu verwenden.
model = XGBClassifier(n_estimators=100, random_state=7, use_label_encoder=False, eval_metric='logloss')

# Cross-Validation
cv = KFold(n_splits=15, random_state=42, shuffle=True)
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
print(f'Accuracy scores for the 10 folds: {scores}')
print(f'Average accuracy: {scores.mean()}')

# Trainiere das Modell mit dem gesamten Trainingsdatensatz
model.fit(X_train, y_train)

# Mache Vorhersagen für den Testdatensatz
X_test = test.drop(['PassengerId', 'Survived'], axis=1)
predictions = model.predict(X_test)

# Füge Vorhersagen in den Testdatensatz ein und exportiere in CSV
test['Survived'] = predictions
result = test[['PassengerId', 'Survived']]
result.to_csv(data_path / '../csv/submission_test.csv', index=False)


891
418
Accuracy scores for the 10 folds: [0.86666667 0.76666667 0.83333333 0.8        0.81666667 0.68333333
 0.89830508 0.84745763 0.89830508 0.79661017 0.77966102 0.81355932
 0.81355932 0.81355932 0.88135593]
Average accuracy: 0.8206026365348398
